In [ ]:
#Jaderná elektrárna Dukovany(두코바니 원자력 발전소)

#https://search.seznam.cz/clanky/?q=Jadern%C3%A1%20elektr%C3%A1rna%20Dukovany
#리눅스 환경에서 코드 실행 중

: 

In [2]:
import time
import json
import datetime
import random
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.01.01"
END_DATE = "2022.03.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()


🔍 검색 중: 두코바니 원전 (2022.01.01 ~ 2022.03.31)
✅ 198개의 뉴스 기사 수집 완료! (2022.01.01 ~ 2022.03.31)


📰 뉴스 본문 크롤링 진행:  24%|██▎       | 47/198 [00:23<00:56,  2.67it/s]

⚠ 오류 발생 (뉴스 크롤링 실패, 1/3): http://dream.kotra.or.kr/kotranews/cms/news/actionKotraBoardDetail.do?SITE_NO=3&MENU_ID=410&CONTENTS_NO=1&bbsGbn=242&bbsSn=242&pNttSn=193341, 오류: HTTPSConnectionPool(host='dream.kotra.or.kr', port=443): Read timed out. (read timeout=10)
⚠ 오류 발생 (뉴스 크롤링 실패, 2/3): http://dream.kotra.or.kr/kotranews/cms/news/actionKotraBoardDetail.do?SITE_NO=3&MENU_ID=410&CONTENTS_NO=1&bbsGbn=242&bbsSn=242&pNttSn=193341, 오류: HTTPSConnectionPool(host='dream.kotra.or.kr', port=443): Read timed out. (read timeout=10)


📰 뉴스 본문 크롤링 진행:  24%|██▍       | 48/198 [00:55<24:40,  9.87s/it]

⚠ HTTP 오류: 404
⚠ HTTP 오류: 404
⚠ HTTP 오류: 404


...

⚠ HTTP 오류: 404
⚠ HTTP 오류: 404
⚠ HTTP 오류: 404


📰 뉴스 본문 크롤링 진행:  99%|█████████▉| 196/198 [03:43<00:01,  1.77it/s]

⚠ HTTP 오류: 404
⚠ HTTP 오류: 404
⚠ HTTP 오류: 404


📰 뉴스 본문 크롤링 진행: 100%|██████████| 198/198 [03:56<00:00,  1.20s/it]

✅ 2022.01.01 ~ 2022.03.31 크롤링 완료! (저장 파일: naver_news_20220101_20220331.json)


In [3]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.04.01"
END_DATE = "2022.06.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2022.04.01 ~ 2022.06.31)
✅ 300개의 뉴스 기사 수집 완료! (2022.04.01 ~ 2022.06.31)


📰 뉴스 본문 크롤링 진행:   8%|▊         | 24/300 [00:10<01:57,  2.34it/s]

⚠ 오류 발생 (뉴스 크롤링 실패, 1/3): http://www.sentv.co.kr/news/view/623229, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/623229 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 2/3): http://www.sentv.co.kr/news/view/623229, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/623229 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 3/3): http://www.sentv.co.kr/news/view/623229, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/623229 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certifi

📰 뉴스 본문 크롤링 진행:  33%|███▎      | 99/300 [00:56<02:10,  1.54it/s]

⚠ 오류 발생 (뉴스 크롤링 실패, 1/3): http://theviewers.co.kr/View.aspx?No=2379232, 오류: HTTPSConnectionPool(host='theviewers.co.kr', port=443): Read timed out. (read timeout=10)


📰 뉴스 본문 크롤링 진행:  90%|█████████ | 270/300 [02:27<00:12,  2.34it/s]

⚠ HTTP 오류: 404
⚠ HTTP 오류: 404
⚠ HTTP 오류: 404


📰 뉴스 본문 크롤링 진행: 100%|██████████| 300/300 [02:52<00:00,  1.74it/s]

✅ 2022.04.01 ~ 2022.06.31 크롤링 완료! (저장 파일: naver_news_20220401_20220631.json)


In [4]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.07.01"
END_DATE = "2022.09.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2022.07.01 ~ 2022.09.31)
✅ 242개의 뉴스 기사 수집 완료! (2022.07.01 ~ 2022.09.31)


📰 뉴스 본문 크롤링 진행: 100%|██████████| 242/242 [01:53<00:00,  2.13it/s]


✅ 2022.07.01 ~ 2022.09.31 크롤링 완료! (저장 파일: naver_news_20220701_20220931.json)


In [5]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.10.01"
END_DATE = "2022.12.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2022.10.01 ~ 2022.12.31)
✅ 150개의 뉴스 기사 수집 완료! (2022.10.01 ~ 2022.12.31)


📰 뉴스 본문 크롤링 진행: 100%|██████████| 150/150 [01:07<00:00,  2.23it/s]

✅ 2022.10.01 ~ 2022.12.31 크롤링 완료! (저장 파일: naver_news_20221001_20221231.json)


In [6]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.01.01"
END_DATE = "2023.03.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2023.01.01 ~ 2023.03.31)
✅ 144개의 뉴스 기사 수집 완료! (2023.01.01 ~ 2023.03.31)


📰 뉴스 본문 크롤링 진행: 100%|██████████| 144/144 [00:55<00:00,  2.59it/s]

✅ 2023.01.01 ~ 2023.03.31 크롤링 완료! (저장 파일: naver_news_20230101_20230331.json)


In [7]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.04.01"
END_DATE = "2023.06.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2023.04.01 ~ 2023.06.31)
✅ 115개의 뉴스 기사 수집 완료! (2023.04.01 ~ 2023.06.31)


📰 뉴스 본문 크롤링 진행: 100%|██████████| 115/115 [00:51<00:00,  2.24it/s]

✅ 2023.04.01 ~ 2023.06.31 크롤링 완료! (저장 파일: naver_news_20230401_20230631.json)


In [8]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.07.01"
END_DATE = "2023.09.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2023.07.01 ~ 2023.09.31)
✅ 162개의 뉴스 기사 수집 완료! (2023.07.01 ~ 2023.09.31)


📰 뉴스 본문 크롤링 진행:  31%|███▏      | 51/162 [00:19<00:48,  2.31it/s]

⚠ 오류 발생 (뉴스 크롤링 실패, 1/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 2/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 3/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certifi

📰 뉴스 본문 크롤링 진행:  40%|███▉      | 64/162 [00:38<00:35,  2.77it/s]

⚠ 오류 발생 (뉴스 크롤링 실패, 1/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 2/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 3/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certifi

...

⚠ 오류 발생 (뉴스 크롤링 실패, 1/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 2/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 3/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certifi

📰 뉴스 본문 크롤링 진행:  94%|█████████▍| 153/162 [02:02<00:02,  3.04it/s]

⚠ 오류 발생 (뉴스 크롤링 실패, 1/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 2/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
⚠ 오류 발생 (뉴스 크롤링 실패, 3/3): http://www.sentv.co.kr/news/view/666666, 오류: HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/666666 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certifi

📰 뉴스 본문 크롤링 진행: 100%|██████████| 162/162 [02:21<00:00,  1.15it/s]

✅ 2023.07.01 ~ 2023.09.31 크롤링 완료! (저장 파일: naver_news_20230701_20230931.json)


In [9]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.10.01"
END_DATE = "2023.12.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2023.10.01 ~ 2023.12.31)
✅ 143개의 뉴스 기사 수집 완료! (2023.10.01 ~ 2023.12.31)


📰 뉴스 본문 크롤링 진행: 100%|██████████| 143/143 [00:55<00:00,  2.56it/s]

✅ 2023.10.01 ~ 2023.12.31 크롤링 완료! (저장 파일: naver_news_20231001_20231231.json)


In [10]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.01.01"
END_DATE = "2024.03.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2024.01.01 ~ 2024.03.31)
✅ 167개의 뉴스 기사 수집 완료! (2024.01.01 ~ 2024.03.31)


📰 뉴스 본문 크롤링 진행: 100%|██████████| 167/167 [01:26<00:00,  1.92it/s]

✅ 2024.01.01 ~ 2024.03.31 크롤링 완료! (저장 파일: naver_news_20240101_20240331.json)


In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.04.01"
END_DATE = "2024.06.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

🔍 검색 중: 두코바니 원전 (2024.04.01 ~ 2024.06.31)


In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.07.01"
END_DATE = "2024.09.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.10.01"
END_DATE = "2024.12.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
import json
import os

# ✅ JSON 파일 리스트 가져오기 (naver_news_20240905_20241231.json 제외)
json_files = [f for f in os.listdir() if f.startswith("naver_news_") and f.endswith(".json") and f != "naver_news_20240905_20241231.json"]

# ✅ 모든 JSON 파일 합치기
all_news = []
for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        news_data = json.load(f)
        all_news.extend(news_data)

# ✅ 통합 JSON 파일 저장
output_file = "naver_news_all.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_news, f, ensure_ascii=False, indent=4)

print(f"✅ {len(all_news)}개의 뉴스 기사를 통합 저장했습니다! (파일명: {output_file})")
